<a href="https://colab.research.google.com/github/Sklan/siamese/blob/master/siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install tf-nightly-2.0-preview

    100% |████████████████████████████████| 80.3MB 357kB/s 
    100% |████████████████████████████████| 61kB 21.4MB/s 
    100% |████████████████████████████████| 358kB 19.5MB/s 
    100% |████████████████████████████████| 3.0MB 1.3MB/s 


In [0]:
import matplotlib.pyplot as plt
from matplotlib import offsetbox

import numpy as np

import tensorflow as tf

import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Layer

from sklearn.preprocessing import normalize

In [0]:
tf.enable_eager_execution()
tf.executing_eagerly() 

True

In [0]:
num_classes = 10
epochs = 500
batch_size = 256
steps = 100

In [0]:
mnist = tfds.load(name="mnist")

In [0]:
train, test = mnist['train'], mnist['test']

In [0]:
train_iterator = train.repeat().shuffle(1024).batch(batch_size).__iter__()

In [0]:
def preprocess(x):
  x = tf.reshape(x, (-1, 784))
  x = tf.cast(x, tf.float32)
  x = tf.divide(x, 255.)
  return x

In [0]:
def contrastive_loss(model, x, y):
  # If you look carefully the implemented formula is slightly different from the Hadsell'06 paper.
  distance = model(x)
  margin = 5.
  similarity = tf.multiply(y, tf.pow(distance, 2))
  dissimilarity = tf.multiply(tf.subtract(1.0, y), tf.pow(tf.maximum(0.0, margin - distance), 2))
  loss = tf.reduce_mean(tf.multiply(tf.add(similarity, dissimilarity), .5)) 
  return loss

In [0]:
def generate_pairs(iterator, steps):
  i = 0
  while i < steps:
    feature = iterator.next()
    x1 = preprocess(feature['image'])
    y1 = feature['label']
    feature = iterator.next()
    x2 = preprocess(feature['image'])
    y2 = feature['label']
    y = tf.cast(tf.equal(y1, y2), tf.float32)
    i = i + 1
    yield [x1, x2], y

In [0]:
def visualize(embed, x_test, y_test):
  # From https://github.com/ywpkwon/siamese_tf_mnist
    feat = embed
    ax_min = np.min(embed,0)
    ax_max = np.max(embed,0)
    ax_dist_sq = np.sum((ax_max - ax_min)**2)

    plt.figure()
    ax = plt.subplot(111)
    colormap = plt.get_cmap('tab10')
    shown_images = np.array([[1., 1.]])
    for i in range(feat.shape[0]):
        dist = np.sum((feat[i] - shown_images)**2, 1)
        if np.min(dist) < 3e-4*ax_dist_sq:   # don't show points that are too close
            continue
        shown_images = np.r_[shown_images, [feat[i]]]
        patch_to_color = np.expand_dims(x_test[i], -1)
        patch_to_color = np.tile(patch_to_color, (1, 1, 3))
        patch_to_color = (1-patch_to_color) * (1,1,1) + patch_to_color * colormap(y_test[i]/10.)[:3]
        imagebox = offsetbox.AnnotationBbox(
            offsetbox.OffsetImage(patch_to_color, zoom=0.5, cmap=plt.cm.gray_r),
            xy=feat[i], frameon=False
        )
        ax.add_artist(imagebox)

    plt.axis([ax_min[0], ax_max[0], ax_min[1], ax_max[1]])
    plt.title('Embedding from the last layer of the network')
    plt.show()


In [0]:
class Siamese(Model):
  def __init__(self):
    super(Siamese, self).__init__()
    self.dense_left1 = layers.Dense(512, activation='relu',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
    self.dense_left2 = layers.Dense(512, activation='relu',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
    self.dense_left3 = layers.Dense(512, activation='relu',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
    self.dense_left4 = layers.Dense(512, activation='relu',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
    self.dense_left5 = layers.Dense(2, 
                                      activation='sigmoid',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
   

    self.dense_right1 = layers.Dense(512, activation='relu',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform') 
    self.dense_right2 = layers.Dense(512, activation='relu',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
    self.dense_right3 = layers.Dense(512, activation='relu',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
    self.dense_right4 = layers.Dense(512, activation='relu',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
    self.dense_right5 = layers.Dense(2, activation='sigmoid',
                                      kernel_initializer='glorot_uniform',
                                      bias_initializer='glorot_uniform')
    
    self.a = None
    self.b = None
    self.x = None
    self.y = None
    
  def call(self, inputs):
    self.a, self.b = inputs
    z = self.dense_left1(self.a)
    z = self.dense_left2(z)
    z = self.dense_left3(z)
    z = self.dense_left4(z)
    self.x = self.dense_left5(z)

    z = self.dense_right1(self.b)
    z = self.dense_right2(z)
    z = self.dense_right3(z)
    z = self.dense_right4(z)
    self.y = self.dense_right5(z)
    
    # Calculate the Euclidean Distance
    distance = tf.sqrt(tf.reduce_sum(tf.pow(tf.subtract(self.x, self.y), 2), axis=1, keepdims=True))
    
    return distance
    

In [0]:
model = Siamese()

In [0]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = contrastive_loss(model, inputs, targets)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [0]:
learning_rate = tf.constant(0.00001)

In [0]:
optimizer = tf.optimizers.SGD(learning_rate, momentum=0.0001, nesterov=True)
global_step = tf.Variable(0)

In [0]:
for epoch in range(epochs):
  x, y = None, None
  for x, y in generate_pairs(train_iterator, steps) :
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables), global_step)
  print('Epoch:', epoch, 'loss:', loss_value.numpy())
  if epoch % 50 == 0:
    em = model.x.numpy()
    x = np.reshape(x[0], (-1, 28, 28))
    visualize(em, x, y)